In [ ]:
from dowhy import CausalModel
import pandas as pd
import data_template, dri
import csv
import networkx
from networkx.drawing.nx_pydot import to_pydot
import score

In [ ]:
# physical traits of user
feet, inches = 5, 7
height = [feet, inches]
age = 30
weight = 170
gender = "male"

In [ ]:
# Initialize Person class and gather data for each recorded day in given csv files
Tom = data_template.create_person(gender,age,height)

# Get daily recommendations for nutrients
a,b,c = dri.pull(gender, age, feet, inches, weight)
macros, vitamins, minerals = dri.process_data(a, b, c)

In [ ]:
# Get average hours of sleep to assign to days with nutrient measurement but no sleep hours
totalSleep, days = 0, 0

with open("noNull/sleep_noNull.csv", 'r') as file:
    reader = csv.reader(file)
    next(reader)

    
    for row in reader:
        totalSleep += float(row[1])
        days += 1

avgSleep = totalSleep / days

In [ ]:
# Get scores for overall health, sleep, macronutrients, and vitamins & minerals
with open("score.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Health Score', 'Sleep score', 'Macro score', 'Vitamin and Mineral Score'])

    count = 0
    for i,v in Tom.Days.items():
        if count >= 20:
            break
        if v.nutrition:
            scoring, sleep, macro, vit_min = score.score([feet, inches], weight, age, gender, avgSleep if not v.sleep else v.sleep, v.nutrition)
            results = [i, scoring, sleep, macro, vit_min]
            writer.writerow(results)
            
            count += 1

In [ ]:
# Organize all scores and measurements into one combined csv file to process together
file, score, nutri, sleep= open("combined.csv", "w"), open('score.csv', 'r'), open('noNull/nutrition_noNull.csv', 'r'), open('noNull/sleep_noNull.csv', 'r')
writer, reader1, reader2, reader3 = csv.writer(file), csv.reader(score), csv.reader(nutri), csv.reader(sleep)

h1, h2, h3 = next(reader1), next(reader2), next(reader3)
head = h1 + [h3[1]] + h2[1:-1]
writer.writerow(head)
sleep.close()

allMacros, allV_M = [], []

count = 0
for i, v in Tom.Days.items():
    if count == 20:
        break
    if v.nutrition:
        vit_min = {}
        vit_min.update(vitamins)
        vit_min.update(minerals)
        if not allMacros and not allV_M:
            allMacros, allV_M = [m for m in macros if m in v.nutrition and macros[m]], [vm for vm in vit_min if vm in v.nutrition and vit_min[vm]]

        scoring, nutrient = next(reader1),next(reader2)
        writer.writerow(scoring + [avgSleep if not v.sleep else v.sleep] + nutrient[1:-1])
        count += 1

file.close()
score.close()
nutri.close()

In [ ]:
df = pd.read_csv('combined.csv')
df.head()

In [ ]:
# Map the measurements to their respective subscores and the subscores to Health score in a DAG
graph = networkx.DiGraph()

graph.add_edge("Macro score", "Health Score")
graph.add_edge("Vitamin and Mineral Score", "Health Score")
graph.add_edge("Sleep score", "Health Score")

graph.add_edges_from([(m, "Macro score") for m in allMacros])
graph.add_edges_from([(vm, "Vitamin and Mineral Score") for vm in allV_M])
graph.add_edge("Sleep(hours)", "Sleep score")


# Convert DAG to string for Causal Model
graph_dot = to_pydot(graph).to_string()

In [ ]:
# Iterate through all macros recorded in csv file and determine their average effect on the Macro score
macro_effect = dict()

for m in allMacros:
    model = CausalModel(data=df, treatment=m, outcome="Macro score", graph=graph_dot)

    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    estimate = model.estimate_effect(identified_estimand,
                                    method_name="backdoor.linear_regression",
                                    test_significance=0.05,
                                    confidence_intervals=True)
    print("----------------------------------------------------------------------------------------------------------------")
    macro_effect[m] = {'avg effect': estimate.value, 'p-value': estimate.test_stat_significance()['p_value'][0]}

In [ ]:
# Display of all recorded macros with their average effect on macro score along with their p-values
macro_effect

In [ ]:
# Iterate through all vitmains and minerals recorded in csv file and determine their average effect on the Vitamin and Mineral score
vm_effect = dict()

for m in allV_M:
    model = CausalModel(data=df, treatment=m, outcome="Vitamin and Mineral Score", graph=graph_dot)

    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    print(identified_estimand)
    estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression",
                                 test_significance=0.05,
                                 confidence_intervals=True)
    print("----------------------------------------------------------------------------------------------------------------")
    vm_effect[m] = {'avg effect': estimate.value, 'p-value': estimate.test_stat_significance()['p_value'][0]}


In [ ]:
# Display of all recorded vitmains and minerals with their average effect on vitmains and minerals score along with their p-values
vm_effect

In [ ]:
blank = None, None, None

def getMinNutrient(effect: dict) -> tuple:
    minEffect = min([effect[n]['avg effect'] for n in effect])

    for n in effect:
        if effect[n]['avg effect'] == minEffect:
            return n, effect[n]['avg effect'],  effect[n]['p-value']

    return blank

In [ ]:
# Determine which of the recorded nutrient has the smallest effect on their respective subscore
minNutrient = blank

minVitMin, minMacro = getMinNutrient(vm_effect), getMinNutrient(macro_effect)
if minVitMin == blank and minMacro != blank:
    minNutrient = minMacro
elif minVitMin != blank and minMacro == blank:
    minNutrient = minVitMin
elif minVitMin != blank and minMacro != blank:
    minNutrient = minVitMin if min(minVitMin[1], minMacro[1]) == minVitMin[1] else minMacro

In [ ]:
# Determine average effect of sleep hours on sleep score
model = CausalModel(data=df, treatment='Sleep(hours)', outcome="Sleep score", graph=graph_dot)
model.view_model()

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
estimate = model.estimate_effect(identified_estimand,
                                method_name="backdoor.linear_regression",
                                test_significance=0.05,
                                confidence_intervals=True)

sleep_effect = "Sleep(hours)", estimate.value, estimate.test_stat_significance()['p_value'][0]

In [ ]:
# Display of average effect of sleep of sleep score and its p-value
sleep_effect

In [ ]:
# Display the nutrient with the smallest effect of on its subscore
minNutrient

In [ ]:

model = CausalModel(data=df, treatment='Sleep score', outcome="Health Score", graph=graph_dot)
model.view_model()

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
estimate = model.estimate_effect(identified_estimand,
                                method_name="backdoor.linear_regression",
                                test_significance=0.05,
                                confidence_intervals=True)

print(estimate)

In [ ]:
model = CausalModel(data=df, treatment='Macro score', outcome="Health Score", graph=graph_dot)
model.view_model()

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
estimate = model.estimate_effect(identified_estimand,
                                method_name="backdoor.linear_regression",
                                test_significance=0.05,
                                confidence_intervals=True)

print(estimate)

In [ ]:
model = CausalModel(data=df, treatment='Vitamin and Mineral Score', outcome="Health Score", graph=graph_dot)
model.view_model()

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
estimate = model.estimate_effect(identified_estimand,
                                method_name="backdoor.linear_regression",
                                test_significance=0.05,
                                confidence_intervals=True)
print(estimate)